### Do the imports
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


### Install pytorch if it isn't installed yeet

Use this website to configure the right installation packages:
[Pytorch](https://pytorch.org/get-started/locally/)

then use `pip install <list of packages>` to install it

For my macbook, it is

```pip3 install torch torchvision torchaudio```



In [2]:
import os
import glob
import sys

import tifffile # read an write tiff files
from cellpose import models
import napari
import torch
import nd2
import numpy as np

from skimage.transform import rescale

import utils

Create the napari viewer to visualize images and mask

In [3]:
viewer = napari.Viewer()

Find the image files to work with. Different path for Window, Mac, and Linux

In [4]:
if sys.platform == 'darwin': 
    imagedir = "/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data"
elif sys.plaform == 'win32':
    imagedir = os.path.join(['S:', 'micro',
                             'internal', 'cjw',
                             'PythonImageProcessing',
                             'cellpose',
                             'Data'])
else:
   imagedir = "/n/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data"
   
imagefiles = sorted(glob.glob(f"{imagedir}/*.tif"))
imagefiles

['/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/01-yeast_many.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/02-a01_42.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/03-a01_43.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/04-nuclei_fish.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/05-yeast_01.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/06-yeast_02.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/fluo_00.tif']

In [7]:
yeast = tifffile.imread(imagefiles[0])
yeast.shape

(2048, 2048)

In [27]:
x = nd2.imread("/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/QS_no-drug-0003.nd2")
x = x.max(axis=0)
x.shape

(3, 2044, 2048)

Add the image to the napari viewer

In [28]:
viewer.layers.clear()
viewer.add_image(x)


<Image layer 'x' at 0x37bd026e0>

### Setup the cellpose model

#### GPU setting
- on windows or linux with an NVIDIA GPU, set `gpu=True``
- on M1 or M2 mac, set `device=torch.device('mps')`
- on old Mac or Windows without GPU, set `gpu=False` -- this will be slower

#### Model
- cellpose has a number of pretrained models to use. Start with `cyto` or `cyto2` (even it is nuclei)
- info about pretrained cellpose [models](https://cellpose.readthedocs.io/en/latest/models.html)

In [11]:
d = torch.device('mps')

# change gpu=True is on windows, and get rid of device
# model = models.Cellpose(gpu=True, model_type='cyto2')
model = models.Cellpose(gpu=False, device=d, model_type='cyto2')

### `model.eval`
Parameters
----------

The basic options:
- x : the image, can be a 2d numpy array, a list of numpy arrays,
or a 3d numpy array
- diameter : The approximate size of the object you are trying to segment
- channels :
    - [0, 0] for a grayscale image
    - [] 

Returns
-------
- masks : An array or list of arrays with segmenation labels/masks
- flows : A list of numpy arrays with fow

In [34]:

masks, flows, styles, diams = model.eval(x[-1], channels=[0, 0],   #[0, 3] might work best
                                         diameter=75)

Show the results from cellpose a layers in napari

In [35]:
viewer.layers.clear()
viewer.add_image(x[-1])
viewer.add_labels(masks)


<Labels layer 'masks' at 0x37b966500>

In [25]:
## Check to see how many yeast cells were found
masks.max()

1913

In [ ]:
viewer.add_image(flows[0], visible=False, name="xyflow")
viewer.add_image(flows[1][0], visible=False, name="xflow")
viewer.add_image(flows[2], visible=False, name="probability")

See what happens when the diameter is changed. What diameter is the best?